In [ ]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = loadmat("products_BAU.mat")

In [ ]:
data.keys()

## Inputs

Total input to each sector

In [ ]:
def flatten2cells(a):
    """Flatten 2 extra dimensions in mat file cell arrays."""
    return np.stack([
        np.stack([
            a[0, i][0, j]
            for j in range(a[0, i].shape[1])
        ])
        for i in range(a.shape[1])
    ])

def flatten3cells(a):
    """Flatten 3 extra dimensions in mat file cell arrays."""
    return np.stack([
        np.stack([
            np.stack([
                a[0, i][0, j][0, k]
                for k in range(a[0, i][0, j].shape[1])
            ])
            for j in range(a[0, i].shape[1])
        ])
        for i in range(a.shape[1])
    ])

In [ ]:
inp = flatten2cells(data["input2sector"])

In [ ]:
inp.shape

8 regions (m), 14 polymers (j), 8 sectors (i), 123 years (k) counting from 1978

In [ ]:
t = np.arange(123) + 1978

In [ ]:
sectors = """\
Packaging
Transportation
Buildings and constructions
Electrical and electronic
Consumer & institutional
Industrial machinery
Textile
Others""".splitlines()

polymers = """\
LDPE
LLDPE
HDPE
PP
PS
PVC
PET
PUR
Polyester fiber
Polyamid fiber
Other fiber
Rubber (7)
Other thermoplastics (6)
Other thermosets (2)
""".splitlines()

regions = """\
North America
South America
Western and Central Europe
Central Europe and Eastern Asia
Africa
Middle East
Northeast Asia
South Asia and Pacific
""".splitlines()

In [ ]:
for i in range(len(sectors)):
    plt.plot(t, inp[0, :, i, :].sum(axis=0), label=sectors[i])
plt.legend();

For each sector the virgin and recycled amounts should sum.

In [ ]:
inp_v = flatten2cells(data["input2sectorvirgin"])
inp_r = flatten2cells(data["input2sectorrecycled"])

In [ ]:
abs((inp_v + inp_r) - inp).max()

Yes -- they agree!

The ratios should be consistent with the input data?

In [ ]:
ratio = flatten2cells(data["ratioPlminSec"])[:, :, 0, :]
ratio.shape
# region, sector, polymer

All sum to 1 over polymers:

In [ ]:
ratio.sum(axis=-1)

All the ratios of `inp` for the year 2019 are consistent with these ratios:

In [ ]:
max([
    abs(inp[m, :, i, 44] / inp[m, :, i, 44].sum() - ratio[m, i, :]).max()
    for m in range(8)
    for i in range(8)
])

How have the ratios changed?

In [ ]:
from ipywidgets import interact
@interact(m=(0, 7), i=(0, 7))
def plot_region_sector_polymer_shares(m, i):
    shares = inp[m, :, i, :] / inp[m, :, i, :].sum(axis=0, keepdims=True)
    plt.stackplot(t, shares, labels=polymers)
    plt.title(f"{regions[m]} | {sectors[i]}")
    plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))

## Waste

In [ ]:
waste = flatten3cells(data["wastetemporary"])
waste.shape
# region, polymer, sector, t_out, t_cohort

In [ ]:
t_grid = np.meshgrid(t, t)
@interact(m=(0, 7), j=(0, 13), i=(0, 7))
def plot_region_sector_polymer_shares(m, j, i):
    plt.title(f"{regions[m]} | {polymers[j]} | {sectors[i]}")
    plt.scatter(t_grid[0].ravel(), t_grid[1].ravel(), waste[m, j, i, :, :].T.ravel() / 3, alpha=0.3)
    plt.ylabel("Cohort year");
    plt.xlabel("Outflow year");

In [ ]:
m = 0
j = 4
i = 2
waste[m, j, i, :, :] / waste[m, j, i, :, :].sum(axis=-1, keepdims=True)

In [ ]:
t_grid = np.meshgrid(t, t)
@interact(m=(0, 7), j=(0, 13), i=(0, 7))
def plot_region_sector_polymer_shares(m, j, i):
    plt.title(f"{regions[m]} | {polymers[j]} | {sectors[i]}")
    #with np.errstate(divide='ignore', invalid='ignore'):
    #    cohort_shares = waste[m, j, i, :, :] / waste[m, j, i, :, :].sum(axis=0, keepdims=True)
    #cohort_shares[0] = np.nan
    plt.contourf(t, t, waste[m, j, i, :, :].T)
    plt.ylabel("Cohort year");
    plt.xlabel("Outflow year");